In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
#from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


In [4]:
df=pd.read_csv('StudentsPerformance.csv')
df.head(4)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44


In [5]:
X=df.drop(columns=['math score'],axis=1)
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
Y=df['math score']
Y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [7]:
num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numarical_transform=StandardScaler()
cat_transforma=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
    ('numarical',numarical_transform,num_features),
    ('categorical',cat_transforma,cat_features)
    ]
)

In [8]:
X=preprocessor.fit_transform(X)

In [9]:
X.shape

(1000, 19)

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=42,test_size=0.2)
X_train.shape,X_test[:,6:7].shape

((800, 19), (200, 1))

In [11]:
def evaluate_model(actual,predict):
    mse=mean_squared_error(actual,predict)
    rmse=np.sqrt(mse)
    r2_square=r2_score(actual,predict)
    return mse,rmse,r2_square




In [44]:
model_dict={
    "linear Regressor":LinearRegression(),
    "lasso":Lasso(),
    "ridge":Ridge(),
    "k-nearest neightbours":KNeighborsRegressor(),
    "DecisionTreeRegressor":DecisionTreeRegressor(),
    "AdaBoostRegressor":AdaBoostRegressor(),
    "XGBoost" : XGBRegressor(),
    "RandomForestRegressor":RandomForestRegressor(),
    

}

model_list=[]
r2_list=[]
i=0
for model_name in model_dict:
    
    models=model_dict[model_name]
    models.fit(X_train,y_train)

    y_train_pre=models.predict(X_train)
    y_test_pre=models.predict(X_test) 

    model_train_mse,model_train_rmse,r2_train_square=evaluate_model(y_train,y_train_pre)
    model_test_mse,model_test_rmse,r2_test_square=evaluate_model(y_test,y_test_pre)

    print(list(model_dict.keys())[i])
    model_list.append(list(model_dict.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mse))
    print("- R2 Score: {:.4f}".format(r2_train_square))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mse))
    print("- R2 Score: {:.4f}".format(r2_test_square))
    r2_list.append(r2_test_square)
    
    print('='*35)
    print('\n')
    i+=1


linear Regressor
Model performance for Training set
- Root Mean Squared Error: 5.3231
- Mean Absolute Error: 28.3349
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3940
- Mean Absolute Error: 29.0952
- R2 Score: 0.8804


lasso
Model performance for Training set
- Root Mean Squared Error: 6.5938
- Mean Absolute Error: 43.4783
- R2 Score: 0.8071
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5197
- Mean Absolute Error: 42.5063
- R2 Score: 0.8253


ridge
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 28.3378
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 29.0563
- R2 Score: 0.8806


k-nearest neightbours
Model performance for Training set
- Root Mean Squared Error: 5.7134
- Mean Absolute Error: 32.6428
- R2 Score: 0.8552
-----------------

In [42]:
tree_model=XGBRegressor()
tree_model.fit(X_train,y_train)

pre=tree_model.predict(X_test)
mse,rmse,r2=evaluate_model(y_test,pre)
mse,rmse,r2

(43.845362371473584, 6.621583071401701, 0.8198173310440375)